In [1]:
import os 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np 

In [2]:
# radar plots - one color for video data, one color for video data 
# is general "gestalt" of home the same as that for the mat data?  

# why stride time, cadence, double and single support ? 
    # stride time and cadence most accurate vid metrics 
    # double and single support from mat most strongly correlated with EDSS and T25FW 

In [3]:
# version 
version = '004'

In [4]:
# home videos 
hv_all_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis',
                           version, 
                           'hv_bw_merged.csv') 
hv_all_df = pd.read_csv(hv_all_path, index_col = 0)

In [5]:
# output path 
output_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis',
                           version, 
                           'radar_plots') 

if not os.path.exists(output_path): 
    os.makedirs(output_path)

if not os.path.exists(os.path.join(output_path, 'home_videos')): 
    os.makedirs(os.path.join(output_path, 'home_videos'))

In [6]:
def radar_plot_mat_vs_vid(categories, values_1, values_2): 
    # Number of variables/categories
    num_vars = len(categories)

    # Compute angle for each category
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]  # Complete the loop

    # Add first value to close the radar plot
    values_1 += values_1[:1]
    values_2 += values_2[:1]

    # Create the plot
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

    # Plot the first dataset
    ax.plot(angles, values_1, color='b', linewidth=2, linestyle='solid', label='Zeno Walkway')
    ax.fill(angles, values_1, color='b', alpha=0.25)

    # Plot the second dataset
    ax.plot(angles, values_2, color='r', linewidth=2, linestyle='solid', label='Video Pose Estimation')
    ax.fill(angles, values_2, color='r', alpha=0.25)

    # Add category labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)

    # For normalized plots - Set the radial axis range and labels
    ax.set_ylim(0, 1)
    ax.set_yticks([.1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0])
    ax.set_yticklabels(['.1', '.2', '.3', '.4', '.5', '.6', '.7', '.8', '.9', '1.0'], color='gray', size=10)

    # Add legend
    ax.legend(loc='best')

    # Show the plot
    plt.close()
    return fig 

In [7]:
# remove outliers from analysis - throws off normalization 
df = hv_all_df

def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

df_out_removed = remove_outliers_iqr(df, ['PWS_stridetimesecmean', 
                                          'PWS_cadencestepsminmean',
                                          'PWS_totaldsupportmean', 
                                          'PWS_singlesupportmean',
                                          'stride_time_mean_sec_pose_hv',
                                          'mean_cadence_step_per_min_pose_hv',
                                          'foot1_double_support_per_mean_pose_hv',
                                          'foot1_single_support_per_mean_pose_hv'
                                         ]) 


In [8]:
# home videos 
categories = ['PWS Stride Time (Mean)',
              'PWS Cadence (Mean)', 
            
              'PWS Double Support (%),',
              'PWS Single Support (%)' 
            ]

for video_i, current_vid in enumerate(df_out_removed['video_id_date_name_pose_hv']): 
    # mat data 
    values_1 = [df.loc[video_i, 'PWS_stridetimesecmean'],
                df.loc[video_i, 'PWS_cadencestepsminmean'], 
              
                df.loc[video_i, 'PWS_totaldsupportmean'], 
                df.loc[video_i, 'PWS_singlesupportmean']] 

    # video data (home videos) 
    values_2 = [df.loc[video_i, 'stride_time_mean_sec_pose_hv'],  
                df.loc[video_i, 'mean_cadence_step_per_min_pose_hv'], 
         
                df.loc[video_i, 'foot1_double_support_per_mean_pose_hv'], 
                df.loc[video_i, 'foot1_single_support_per_mean_pose_hv']
                ]
                                      
    # normalize mat values vs min and max of all mat values for that metric 
    min_mat_values = [df['PWS_stridetimesecmean'].min(),
                      df['PWS_cadencestepsminmean'].min(),
                     
                      df['PWS_totaldsupportmean'].min(),
                      df['PWS_singlesupportmean'].min()
                      ]
                          
    max_mat_values = [df['PWS_stridetimesecmean'].max(),
                      df['PWS_cadencestepsminmean'].max(),
              
                      df['PWS_totaldsupportmean'].max(), 
                      df['PWS_singlesupportmean'].max()
                     ]
                     
    
     # video min and max 
    min_vid_values = [df['stride_time_mean_sec_pose_hv'].min(),
                      df['mean_cadence_step_per_min_pose_hv'].min(),
                     
                      df['foot1_double_support_per_mean_pose_hv'].min(), 
                      df['foot1_single_support_per_mean_pose_hv'].min()
                  ]

    max_vid_values = [df['stride_time_mean_sec_pose_hv'].max(),
                      df['mean_cadence_step_per_min_pose_hv'].max(),
               
                      df['foot1_double_support_per_mean_pose_hv'].max(),
                      df['foot1_single_support_per_mean_pose_hv'].max()
                      ]


    values_1_normalized = [(v - min_val) / (max_val - min_val) for v, min_val, max_val in zip(values_1, min_mat_values, max_mat_values)]
    values_2_normalized = [(v - min_val) / (max_val - min_val) for v, min_val, max_val in zip(values_2, min_vid_values, max_vid_values)]

    # radar plot 
    current_fig = radar_plot_mat_vs_vid(categories, values_1_normalized, values_2_normalized)
    current_fig.savefig(os.path.join(output_path, 'home_videos', f'hv_{current_vid}_radar_plot.png'), 
                       bbox_inches='tight', dpi=300)